In [3]:
pip install opencv-python

  Using cached opencv_python-4.8.1.78-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (19 kB)
Using cached opencv_python-4.8.1.78-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (61.7 MB)
Note: you may need to restart the kernel to use updated packages.


In [70]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def histogram_equalization(img):
    # Calculate histogram of input image
    hist, _ = np.histogram(img.flatten(), bins=256, range=[0,256])
    
    # Compute cumulative distribution function
    cdf = hist.cumsum()
    
    # Normalize the cdf
    cdf_normalized = (cdf - cdf.min()) * 255 / (cdf.max() - cdf.min())
    
    # Interpolate the cdf
    img_equalized = cdf_normalized[img]
    
    return img_equalized.astype(np.uint8)


def compute_LBP(img, window_size):
    # Function to compute LBP feature descriptor for each window in the image
    
    # Get image dimensions
    height, width = img.shape
    
    # Initialize variables for LBP parameters
    radius = 1  # LBP radius
    n_points = 8 * radius  # Number of neighbor points
    
    # Initialize histogram bins (assuming LBP codes range from 0 to 255)
    bins = np.arange(0, 256, 1)
    
    window_hist = []
    windows = []
    
    # Divide the image into windows
    for y in range(0, height - window_size + 1, window_size):
        for x in range(0, width - window_size + 1, window_size):
            # if window_count in selected_windows:
            window = img[y:y+window_size, x:x+window_size]
            
            # Initialize histogram for the window
            hist = np.zeros((256,), dtype=int)
            
            # Compute LBP for each pixel in the window
            for i in range(1, window_size - 1):
                for j in range(1, window_size - 1):
                    center = window[i, j]
                    
                    # Compare center pixel to its 8 neighbors
                    values = [
                        window[i-1, j-1], window[i-1, j], window[i-1, j+1],
                        window[i, j+1], window[i+1, j+1], window[i+1, j],
                        window[i+1, j-1], window[i, j-1]
                    ]
                    
                    # Compute LBP binary code
                    code = 0
                    for idx, val in enumerate(values):
                        #It shifts the binary digit 1 to the left by the value of idx
                        code += (val >= center) * (1 << idx)
                    
                    # Convert binary code to decimal
                    code_decimal = int(bin(code), 2)
                    
                    # Increment histogram bin for this LBP code
                    hist[code_decimal] += 1

            
            # Normalize histogram
            hist = hist / np.sum(hist)
            window_hist.append(hist)
            windows.append(window)

    return window_hist, windows

def show_windows(window_hist, windows):
    window_count = 0
    for i, window in enumerate(windows):
        window_count += 1
        #Plot original window
        plt.figure(figsize=(10, 4))
        plt.subplot(131)
        plt.axis('off')
        plt.imshow(window, cmap='gray')
        plt.title(f"Window {window_count}")

        #Plot Equalized LBP window
        plt.subplot(132)
        plt.axis('off')
        plt.imshow(histogram_equalization(window), cmap='gray')
        plt.title("Equalized LBP of Window")
        
        #Plot LBP Histogram
        plt.subplot(133)
        plt.bar(range(len(window_hist[i])), window_hist[i])
        plt.title("LBP Histogram (Bar)")
        
        plt.tight_layout()
        plt.show()

In [ ]:
# Define window size (adjust as needed)
window_size = 80

# Call the function to compute LBP for the image
image = cv2.imread("../Dataset/DatasetA/face-3.jpg", cv2.IMREAD_GRAYSCALE)
face_hist, face_window = compute_LBP(image, window_size)

face_descriptor = []
face_descriptor.append(face_hist[2])
face_descriptor.append(face_hist[4])
face_descriptor.append(face_hist[7])


car = cv2.imread("../Dataset/DatasetA/car-3.jpg", cv2.IMREAD_GRAYSCALE)
car_hist, car_window = compute_LBP(car, window_size)

car_descriptor = []
car_descriptor.append(car_hist[0])
car_descriptor.append(car_hist[3])
car_descriptor.append(car_hist[6])

In [ ]:
image1 = cv2.imread("../Dataset/DatasetA/face-1.jpg", cv2.IMREAD_GRAYSCALE)
image2 = cv2.imread("../Dataset/DatasetA/face-3.jpg", cv2.IMREAD_GRAYSCALE)
car3 = cv2.imread("../Dataset/DatasetA/car-1.jpg", cv2.IMREAD_GRAYSCALE)
car4 = cv2.imread("../Dataset/DatasetA/car-2.jpg", cv2.IMREAD_GRAYSCALE)
new_images = []
new_images.append(image1)
new_images.append(image2)
new_images.append(car3)
new_images.append(car4)


def histogram_intersection(hist1, hist2):
    # Compute histogram intersection between two histograms
    return np.sum(np.minimum(hist1, hist2))

def classification(new_images, face_descriptors, car_descriptors):
    for new_image in new_images:
        # compute_LBP
        new_hist, new_window = compute_LBP(new_image, window_size)
        
        # Compare the new global descriptor with face and car descriptors using histogram intersection
        face_similarity_scores =[]
        for face_desc in face_descriptors:
            for hist in new_hist:
                face_similarity_score = histogram_intersection(face_desc, hist)
                face_similarity_scores.append(face_similarity_score)
        car_similarity_scores =[]
        for car_desc in car_descriptors:
            for hist in new_hist:
                car_similarity_score = histogram_intersection(car_desc, hist)
                car_similarity_scores.append(car_similarity_score)
        # Calculate highest similarity scores for faces and cars
        face_similarity_score = np.max(face_similarity_scores)      
        car_similarity_score = np.max(car_similarity_scores)
        
        # Classify the new image based on similarity scores
        if face_similarity_score > car_similarity_score:
            print("The new image is classified as a face.")
        else:
            print("The new image is classified as a car.")

classification(new_images, face_descriptor, car_descriptor)

The new image is classified as a face.
The new image is classified as a face.
The new image is classified as a car.
The new image is classified as a car.


In [160]:
window_size = 50
s_face_hist, s_face_window = compute_LBP(image, window_size)
s_car_hist, s_car_window = compute_LBP(car, window_size)

s_face_histograms =[]
s_face_histograms.append(s_face_hist[8])
s_face_histograms.append(s_face_hist[17])
s_face_histograms.append(s_face_hist[22])

s_car_histograms= []
s_car_histograms.append(s_face_hist[1])
s_car_histograms.append(s_face_hist[14])
s_car_histograms.append(s_face_hist[20])

classification(new_images, s_face_histograms, s_car_histograms)

The new image is classified as a face.
The new image is classified as a face.
The new image is classified as a car.
The new image is classified as a car.


In [164]:
window_size = 100
l_face_hist, l_face_window = compute_LBP(image, window_size)
l_car_hist, l_car_window = compute_LBP(car, window_size)

l_face_histograms =[]
l_face_histograms.append(l_face_hist[0])
l_face_histograms.append(l_face_hist[1])
l_face_histograms.append(s_face_hist[3])

l_car_histograms= []
l_car_histograms.append(l_face_hist[0])
l_car_histograms.append(l_face_hist[2])
l_car_histograms.append(s_face_hist[3])

classification(new_images, l_face_histograms, l_car_histograms)

The new image is classified as a face.
The new image is classified as a face.
The new image is classified as a face.
The new image is classified as a face.
